Note 1: this guide currently omits any mention of notebook testing; that will come in a future version.

Note 2: this is a pre-release version of nbsite, so you may subsequently have to change your config on upgrading to a newer version of nbsite. You may also encounter problems and/or limitations - please file issues or ask questions on [GitHub](https://github.com/pyviz/nbsite/issues).

## Prerequisites

1. You want a 'static', fairly self contained site (i.e. just an ordinary web server is required - you can build and serve locally, or copy the build directory to a remote server).

2. You have one or more sets of notebooks (e.g. `notebooks/`, or `examples/getting_started` + `examples/user_guide` + `examples/gallery`). Note: current expectation is for there to be no title in the notebook itself; instead, the title will be taken from filename. E.g. `1_The_Filename.ipynb` will result in a title of `1 The Filename`. Additionally, notebooks should use level 2 headings as their highest heading level (level 1 will used for the title). However, all of this may change - see [issue 18](https://github.com/pyviz/nbsite/issues/18).

3. The packages used by the notebooks are already installed in the environment into which you will install nbsite.

4. (optional, for API docs) Any packages for which you want to generate API docs are also installed (which could be via e.g. `pip install -e /path/to/pkg`).

5. You are using mac or linux (might work on windows, but currently untested)

6. You are using holoviews for visualization (other packages might work, or no having holoviews might work, but currently untested; if you don't have holoviews, you will at least need to override `nbbuild_ipython_startup` in `conf.py`, setting it to e.g. `""`).

## Installation

`conda install -c pyviz nbsite sphinx_ioam_theme` or `pip install nbsite sphinx_ioam_theme` (note: certain operations may require extra dependencies; for now, see setup.py to learn what those are)

## One time setup

You can skip and/or modify various steps below depending on what you already have. If you do skip a step, you may need to add something to one of your existing files (should be able to figure that out by looking at the corresponding file in `nbsite/tmplate`; e.g. if you have existing conf.py you will need to copy in relevant content from nbsite/tmplate/conf.py).

1. Make a doc dir: `mkdir doc && cd doc`

2. Create a set of basic pages: `nbsite_from_tmplate.py .` (will not overwrite existing files)

3. Edit conf.py as appropriate for your project.

4. Edit doc/index.rst toctree to match pages in conf.py

5. Edit/delete other pages as desired, or add pages. Be sure to update toctrees in index pages as you do this.

5. Commit initial version of your pages: `git commit -m "something" FAQ.rst about.rst conf.py index.rst latest_news.html ...` (etc)

## Building the site

0. `export MODULE=yourmodule`, `export ORG=yourorg`, `export PROJECT=yourproject` (or just replace as appropriate in commands below)

1. *(optional, if you want API docs)* Generate API docs: `nbsite_generate_modules.py ${MODULE} -d ./Reference_Manual -n ${PROJECT} -e tests` (assuming you have `tests` and do not want it to appear in the API docs)

2. Generate rst containers for notebooks (assuming they are in `../examples`: `nbsite_nbpagebuild.py ${ORG} ${PROJECT} ../examples .` 

3. *(optional, if you want gallery)* Make the gallery: `nbsite_gallery.py ../`

4. Make site: `sphinx-build -b html . ./_build/html`

6. Update inter-notebook links: `nbsite_fix_links.py _build/html`

7. Do your notebooks need to access local assets such as images? If so, e.g. `cp -r ../examples/assets _build/html/`.

8. View the result in a browser at localhost:8000: `pushd _build/html && python -m http.server`

## Re-building/updating the site

### rst pages

* Added, removed, or edited an rst page? Update any toc as necessary (e.g. removing an entry), then re-run step 5 above.

### Notebooks

* Removed a notebook? You have to manually delete the corresponding rst container file.

* Edited a notebook? Re-run step 5 above. It ought to only re-generate what it needs to (i.e. what's changed), but that probably doesn't work quite yet. You may need to manually remove a 'temp evaluated' notebook and corresponding html file (both from `_build/html/path/to/notebook`) to have a notebook re-run and turned into html. If you haven't edited the notebook but want it to re-run for another reason e.g. changed dependencies, then you will also need to `touch` the source notebook.

* Added a new notebook? Re-run step 3 then proceed as for 'edited', above.                                                                                   
                                                                                   
### Reference manual

* If you are working on the reference manual, TODO (depends on commit or not).


## Customizing the site

You can customize the style of the site, and you can customize how the IPython notebooks are run.

### Style

The `html_theme` is `sphinx_ioam_theme`. You may be able to use a different theme (untested). However, you can set the following `html_theme_options` in conf.py:

* `logo` and `favicon`: provide paths relative to `html_static_path` (`doc/_static` by default)
* `custom_css`: path relative to `html_static_path` overriding styles. Styles come first from the theme's `main.css` (see [theme issue](https://github.com/ioam/sphinx_ioam_theme/issues/1)), which is extended/overridden by nbsite's own css `nbsite/_shared_static/nbsite.css`, which can then be further extended/overridden by your site's own css.
* js TODO

### Running of IPython notebooks

In conf.py, you can set options to control notebook execution: 

* `nbbuild_cell_timeout`: timeout per cell (seconds), e.g. `100`
* `nbbuild_ipython_startup`: code (as string) to execute before running the first cell of each notebook. Defaults to [nbsite's ipython startup code](https://github.com/pyviz/nbsite/blob/master/nbsite/ipystartup.py). E.g. `"module.special_swith=False"`. 
* `nbbuild_patterns_to_take_along`: list of glob patterns to match files that should be copied alongside a notebook. E.g. holoviews is configured to save data in external json files to improve page loading times, so this defaults to `["*.json"]`.

## Automatic deployment to github pages

0. Remove unnecessary files: `nbsite_cleandisthtml.py ./_build/html take_a_chance`

1. Bypass Jekyll on github pages: `touch ./_build/html/.nojekyll`

2. Add something like this to your `.travis.yml` file (but modify for your own preferences):

```
deploy:
  provider: pages
  skip_cleanup: true
  github_token: $GITHUB_TOKEN
  local_dir: ./_build/html
  on:
    branch: master
```

See nbsite's `.travis.yml` for an example.